In [1]:
### collect images using cv2 and own webcam

In [2]:
import os
import time
import uuid
import cv2
import subprocess

In [3]:
IMAGES_PATH = os.path.join('dataset', 'images')
number_images = 40

In [4]:
"""
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open camera")
    exit()

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 15)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))

time.sleep(2)

for imgnum in range(number_images):
    print(f'Collecting image {imgnum}')
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        continue 

    imgname = os.path.join(IMAGES_PATH, f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)
    
    try:
        cv2.imshow('frame', frame)
    except cv2.error as e:
        print(f"OpenCV error: {e}")

    time.sleep(0.2)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
"""

'\ncap = cv2.VideoCapture(0)\n\nif not cap.isOpened():\n    print("Cannot open camera")\n    exit()\n\ncap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)\ncap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)\ncap.set(cv2.CAP_PROP_FPS, 15)\ncap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(\'M\', \'J\', \'P\', \'G\'))\n\ntime.sleep(2)\n\nfor imgnum in range(number_images):\n    print(f\'Collecting image {imgnum}\')\n    ret, frame = cap.read()\n    if not ret:\n        print("Failed to grab frame")\n        continue \n\n    imgname = os.path.join(IMAGES_PATH, f\'{str(uuid.uuid1())}.jpg\')\n    cv2.imwrite(imgname, frame)\n    \n    try:\n        cv2.imshow(\'frame\', frame)\n    except cv2.error as e:\n        print(f"OpenCV error: {e}")\n\n    time.sleep(0.2)\n\n    if cv2.waitKey(1) & 0xFF == ord(\'q\'):\n        break\n\ncap.release()\ncv2.destroyAllWindows()\n'

In [5]:
import matplotlib
matplotlib.use('TkAgg')

In [6]:
#!labelme
### creating the labels for the pictures(adding a box around the target object in each photo and assign it to a class)

In [7]:
### reviewing te data and building the loading function

In [8]:
import tensorflow as tf
import cv2
import json
import numpy as np
from matplotlib import pyplot as plt

In [9]:
gpus = tf.config.list_physical_devices('GPU')
gpus

I0000 00:00:1721783741.470079    1964 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1721783741.611325    1964 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1721783741.611357    1964 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [11]:
images = tf.data.Dataset.list_files('dataset/images/*.jpg', shuffle = False)

I0000 00:00:1721783741.628685    1964 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1721783741.628742    1964 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1721783741.628753    1964 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1721783742.201385    1964 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1721783742.201434    1964 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: dataset/images/*.jpg'

In [12]:
images.as_numpy_iterator().next()

NameError: name 'images' is not defined

In [13]:
def load_image(x):
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [14]:
images = images.map(load_image)

NameError: name 'images' is not defined

In [15]:
images.as_numpy_iterator().next()

NameError: name 'images' is not defined

In [16]:
image_generator = images.batch(4).as_numpy_iterator()

NameError: name 'images' is not defined

In [17]:
plot_images = image_generator.next()

NameError: name 'image_generator' is not defined

In [18]:
fig, ax = plt.subplots(ncols = 4, figsize = (20, 20))
for idx, image in enumerate(plot_images):
    ax[idx].imshow(image)
plt.show()

NameError: name 'plot_images' is not defined

In [19]:
### Partition unaugmented data train, validation and test
### 140 images for train
### 30 images for val
### 30 images for test
### 112 train
### 26 val
### 26 test

In [20]:
for folder in ['train', 'test', 'val']:
    for file in os.listdir(os.path.join('dataset',folder, 'images')):
        filename = file.split('.')[0]+'.json'
        existing_filepath = os.path.join('dataset', 'labels', filename)
        if os.path.exists(existing_filepath):
            new_filepath = os.path.join('dataset', folder, 'labels', filename)
            os.replace(existing_filepath, new_filepath)

In [21]:
import albumentations as alb

In [22]:
augmentor = alb.Compose([alb.RandomCrop(width = 450, height = 450),
                         alb.HorizontalFlip(p = 0.5),
                         alb.RandomBrightnessContrast(p = 0.2),
                         alb.RandomGamma(p = 0.2),
                         alb.RGBShift(p = 0.2),
                         alb.VerticalFlip(p = 0.5)],
                         bbox_params = alb.BboxParams(format = 'albumentations', label_fields = ['class_labels']))

In [23]:
img = cv2.imread(os.path.join('dataset', 'train', 'images', '7a501311-47e0-11ef-ae67-91e12ce543a5.jpg'))

In [24]:
with open(os.path.join('dataset', 'train', 'labels', '7a501311-47e0-11ef-ae67-91e12ce543a5.json'), 'r') as f:
    label = json.load(f)

In [25]:
label

{'version': '5.5.0',
 'flags': {},
 'shapes': [{'label': 'face',
   'points': [[195.34722222222217, 60.31249999999998],
    [431.80555555555554, 338.09027777777777]],
   'group_id': None,
   'description': '',
   'shape_type': 'rectangle',
   'flags': {},
   'mask': None}],
 'imagePath': '../images/7a501311-47e0-11ef-ae67-91e12ce543a5.jpg',
 'imageData': '...',
 'imageHeight': 480,
 'imageWidth': 640}

In [26]:
coords = [0, 0, 0, 0]
coords[0] = label['shapes'][0]['points'][0][0]
coords[1] = label['shapes'][0]['points'][0][1]
coords[2] = label['shapes'][0]['points'][1][0]
coords[3] = label['shapes'][0]['points'][1][1]
### coords[0] is x1 
### coords[1] is y1
### coords[2] is x2
### coords [3] is y2
### x1, y1 ->top left
### x2, y2 -> bottom right

In [27]:
coords = list(np.divide(coords, [640, 480, 640, 480]))
### the x coordinates get divided by  640 because that's the image width and 
### the y coordinates get divided by 480 because that's the image height
coords

[0.30523003472222215,
 0.12565104166666663,
 0.6746961805555556,
 0.7043547453703703]

In [28]:
augmented = augmentor(image = img, bboxes = [coords], class_labels = ['face'])

In [29]:
augmented['bboxes']

[(0.4270987654320988, 0.286466049382716, 0.9525617283950618, 0.90375)]

In [30]:
cv2.rectangle(augmented['image'],
             tuple(np.multiply(augmented['bboxes'][0][:2], [450, 450]).astype(int)),
             tuple(np.multiply(augmented['bboxes'][0][2:], [450, 450]).astype(int)),
             (255, 0, 0),2)
plt.imshow(augmented['image'])

In [31]:
### Build and run augmentation pipeline

In [32]:
"""
for partition in ['train','test','val']: 
    for image in os.listdir(os.path.join('dataset', partition, 'images')):
        img = cv2.imread(os.path.join('dataset', partition, 'images', image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join('dataset', partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [640,480,640,480]))

        try: 
            for x in range(60):
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])
                cv2.imwrite(os.path.join('aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 


                with open(os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)
"""            

'\nfor partition in [\'train\',\'test\',\'val\']: \n    for image in os.listdir(os.path.join(\'dataset\', partition, \'images\')):\n        img = cv2.imread(os.path.join(\'dataset\', partition, \'images\', image))\n\n        coords = [0,0,0.00001,0.00001]\n        label_path = os.path.join(\'dataset\', partition, \'labels\', f\'{image.split(".")[0]}.json\')\n        if os.path.exists(label_path):\n            with open(label_path, \'r\') as f:\n                label = json.load(f)\n\n            coords[0] = label[\'shapes\'][0][\'points\'][0][0]\n            coords[1] = label[\'shapes\'][0][\'points\'][0][1]\n            coords[2] = label[\'shapes\'][0][\'points\'][1][0]\n            coords[3] = label[\'shapes\'][0][\'points\'][1][1]\n            coords = list(np.divide(coords, [640,480,640,480]))\n\n        try: \n            for x in range(60):\n                augmented = augmentor(image=img, bboxes=[coords], class_labels=[\'face\'])\n                cv2.imwrite(os.path.join(\'aug_d

In [33]:
train_images = tf.data.Dataset.list_files('aug_data/train/images/*.jpg', shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (120,120)))
train_images = train_images.map(lambda x: x/255)

In [34]:
test_images = tf.data.Dataset.list_files('aug_data/test/images/*.jpg', shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (120,120)))
test_images = test_images.map(lambda x: x/255)

In [35]:
val_images = tf.data.Dataset.list_files('aug_data/val/images/*.jpg', shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (120,120)))
val_images = val_images.map(lambda x: x/255)

In [36]:
train_images.as_numpy_iterator().next()

array([[[0.08419117, 0.18082108, 0.08223039],
        [0.09595589, 0.18743873, 0.08400735],
        [0.09540441, 0.16844362, 0.07922794],
        ...,
        [0.028125  , 0.11096814, 0.00851716],
        [0.01960784, 0.10980392, 0.00490196],
        [0.02181373, 0.10741422, 0.00300245]],

       [[0.10392157, 0.17352942, 0.07156863],
        [0.10042892, 0.17218137, 0.06378677],
        [0.09454657, 0.15729167, 0.05189951],
        ...,
        [0.02303922, 0.11323529, 0.00343137],
        [0.02395833, 0.1038603 , 0.00091912],
        [0.03584559, 0.1216299 , 0.01329657]],

       [[0.09405638, 0.16464461, 0.07052696],
        [0.11868872, 0.17898284, 0.09515931],
        [0.10533088, 0.17591912, 0.08265931],
        ...,
        [0.02702206, 0.11721814, 0.00349265],
        [0.02401961, 0.10392157, 0.00098039],
        [0.02095588, 0.1148897 , 0.003125  ]],

       ...,

       [[0.3071691 , 0.42873776, 0.40735295],
        [0.28118873, 0.41335785, 0.37616423],
        [0.22040442, 0

In [37]:
### Prepare labels

In [38]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = 'utf-8') as f:
        label = json.load(f)

    return [label['class']], label['bbox']

In [39]:
def set_shapes(label_class, label_bbox):
    label_class.set_shape([1])
    label_bbox.set_shape([4])
    return label_class, label_bbox

In [40]:
train_labels = tf.data.Dataset.list_files('aug_data/train/labels/*.json', shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))
train_labels = train_labels.map(set_shapes)

In [41]:
test_labels = tf.data.Dataset.list_files('aug_data/test/labels/*.json', shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))
test_labels = test_labels.map(set_shapes)

In [42]:
val_labels = tf.data.Dataset.list_files('aug_data/val/labels/*.json', shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))
val_labels = val_labels.map(set_shapes)

In [43]:
train_labels.as_numpy_iterator().next()

(array([1], dtype=uint8),
 array([0.2607, 0.4072, 0.6543, 0.891 ], dtype=float16))

In [44]:
### Combine images and labels

In [45]:
len(train_images), len(train_labels), len(test_images), len(test_labels), len(val_images), len(val_labels)

(15120, 15120, 3360, 3360, 3360, 3360)

In [46]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(10000)
train = train.batch(32)
train = train.prefetch(4)

In [47]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(2000)
test = test.batch(32)
test = test.prefetch(4)

In [48]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(2000)
val = val.batch(32)
val = val.prefetch(4)

In [49]:
train.as_numpy_iterator().next()[1]

(array([[0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1]], dtype=uint8),
 array([[0.     , 0.     , 0.     , 0.     ],
        [0.2783 , 0.0945 , 0.7393 , 0.7656 ],
        [0.1137 , 0.2957 , 0.6274 , 0.9185 ],
        [0.358  , 0.04422, 0.911  , 0.9067 ],
        [0.     , 0.     , 0.     , 0.     ],
        [0.     , 0.     , 0.     , 0.     ],
        [0.09796, 0.03265, 0.703  , 0.8364 ],
        [0.     , 0.     , 0.     , 0.     ],
        [0.2722 , 0.0307 , 0.7505 , 0.6665 ],
        [0.     , 0.     , 0.     , 0.     ],
        [0.1439 , 0.1627 , 0.515  , 0.689  ],
        [0.     , 0.     , 0.     , 0.     ],
        [0.2712 ,

In [59]:
data_samples = train.as_numpy_iterator()

In [60]:
res = data_samples.next()

In [61]:
fig, ax = plt.subplots(ncols=4, figsize=(20, 20))

for idx in range(4):
    sample_image = res[0][idx].copy()  # Make a writable copy of the image
    sample_coords = res[1][1][idx]

    cv2.rectangle(
        sample_image,
        tuple(np.multiply(sample_coords[:2], [120, 120]).astype(int)),
        tuple(np.multiply(sample_coords[2:], [120, 120]).astype(int)),
        (255, 0, 0), 2
    )

    ax[idx].imshow(sample_image)

plt.show()

In [62]:
## Building the model

In [63]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.regularizers import l2

In [64]:
vgg = VGG16(include_top = False)

In [65]:
vgg.summary()
### stops after the last max pooling layer

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [66]:
def build_model():
    input_layer = Input(shape = (120, 120, 3))
    
    vgg = VGG16(include_top = False)(input_layer)
    
    # Classification model
    # GlobalMaxPooling of the vgg model keeps only the max 512 values of the VGG
    # ( the result of the last max pooling)
    # max 512 from the vgg -> f1 -> 2048 neurons with activation relu
    # -> 1 neuron output layer (value between 0 and 1)

    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation = 'relu', kernel_regularizer=l2(0.01))(f1)
    class1 = Dropout(0.3)(class1)
    class2 = Dense(1, activation = 'sigmoid')(class1)
    
    # Regression model for the bounding box
    # GlobalMaxPooling of the vgg model keeps only the max 512 values of the VGG
    # (the result of the last max pooling)
    # max 512 -> f2 -> 2048 neurons with activ. relu 
    # -> 4 neurons output layer (representing the coordinateso f the box)
    
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation = 'relu', kernel_regularizer=l2(0.01))(f2)
    regress1 = Dropout(0.4)(regress1)
    regress2 = Dense(4, activation = 'sigmoid')(regress1)
    
    facetracker = Model(inputs = input_layer, outputs = [class2, regress2])
    ### the output structure is similar to the labels
    ### e.g [[1], [0.21321, 0.2321, 0.66767, 0,8988]]
    ### class 2 is used to classify the object( value between 0 and 1)
    ### 0 not face 1 face
    ### regress2 has 4 neurons (the coordinates of the box)
    return facetracker

In [67]:
facetracker = build_model()

In [68]:
facetracker.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 120, 120,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vgg16 (Functional)  │ (None, 3, 3, 512) │ 14,714,688 │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 512)       │          0 │ vgg16[0][0]       │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 512)       │          0 │ vgg16[0][0]       │
│ (GlobalMaxPooling2… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 2048)      │  1,050,624 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 2048)      │  1,050,624 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 2048)      │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 2048)      │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │      2,049 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 4)         │      8,196 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,826,181 (64.19 MB)

 Trainable params: 16,826,181 (64.19 MB)

 Non-trainable params: 0 (0.00 B)

In [69]:
X, y = train.as_numpy_iterator().next()

In [70]:
X

array([[[[0.50441176, 0.48480392, 0.5083333 ],
         [0.5204044 , 0.50079656, 0.51648283],
         [0.5156863 , 0.49607843, 0.5117647 ],
         ...,
         [0.09785539, 0.10177696, 0.11746324],
         [0.10765931, 0.10196079, 0.12138481],
         [0.10330883, 0.09693628, 0.11746324]],

        [[0.50147057, 0.48186275, 0.50539213],
         [0.5057598 , 0.48633578, 0.50980395],
         [0.5054534 , 0.4976103 , 0.5172181 ],
         ...,
         [0.07751226, 0.09319853, 0.0971201 ],
         [0.09166667, 0.09705883, 0.10441177],
         [0.08180147, 0.08688726, 0.10189951]],

        [[0.5056985 , 0.4860907 , 0.51354164],
         [0.5127451 , 0.49460784, 0.5176471 ],
         [0.5068015 , 0.49895832, 0.5185662 ],
         ...,
         [0.07015932, 0.09368873, 0.09368873],
         [0.07352941, 0.09019608, 0.09362745],
         [0.07775735, 0.09344362, 0.10226715]],

        ...,

        [[0.2720588 , 0.20049019, 0.27745098],
         [0.2677696 , 0.19865195, 0.2844363 ]

In [71]:
y

(array([[0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1]], dtype=uint8),
 array([[0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      ],
        [0.06085 , 0.06415 , 0.631   , 0.7495  ],
        [0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      ],
        [0.1451  , 0.296   , 0.6     , 0.8687  ],
        [0.5376  , 0.3054  , 0.87    , 0.7783  ],
        [0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      ],
        [0.5024  , 0.0828  , 0.935   , 0.6733  ],
        [0.2196  , 0.3403  , 0.675   , 0.913   ],
        [0.      , 

In [72]:
len(train)

473

In [73]:
### Defining loss functions & optimizers

In [74]:
batches_per_epoch = len(train)
lr_decay = (1./0.75 -1)/batches_per_epoch

In [75]:
optimizer = tf.keras.optimizers.Adam(decay = lr_decay)

/home/sebastian/miniconda3/envs/tf_env/lib/python3.9/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [76]:
def localization_loss(y_true, yhat):            
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2]))
                  
    h_true = y_true[:,3] - y_true[:,1] 
    w_true = y_true[:,2] - y_true[:,0] 

    h_pred = yhat[:,3] - yhat[:,1] 
    w_pred = yhat[:,2] - yhat[:,0] 
    
    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    
    return delta_coord + delta_size

In [77]:
classloss = tf.keras.losses.BinaryCrossentropy()

In [78]:
regressloss = localization_loss

In [79]:
### Training the model

In [80]:
class FaceTracker(Model): 
    def __init__(self, eyetracker,  **kwargs): 
        super().__init__(**kwargs)
        self.model = eyetracker

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt
    
    def train_step(self, batch, **kwargs): 
        
        X, y = batch
        
        with tf.GradientTape() as tape: 
            classes, coords = self.model(X, training=True)
            
            batch_classloss = self.closs(y[0], classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
            
            total_loss = batch_localizationloss+0.5*batch_classloss
            
            grad = tape.gradient(total_loss, self.model.trainable_variables)
        
            self.opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
    
    def test_step(self, batch, **kwargs): 
        X, y = batch
        
        classes, coords = self.model(X, training=False)
        
        batch_classloss = self.closs(y[0], classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
        total_loss = batch_localizationloss+0.5*batch_classloss
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
        
    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)

In [81]:
model = FaceTracker(facetracker)

In [82]:
model.compile(optimizer, classloss, regressloss)

In [83]:
logdir = 'logs'

In [84]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)

In [85]:
##early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_total_loss', patience=5, restore_best_weights=True, mode = 'min')

In [86]:
hist = model.fit(train, epochs = 20, validation_data = val, 
                            callbacks = [tensorboard_callback])

Epoch 1/20


I0000 00:00:1721783871.851350    2129 service.cc:146] XLA service 0x7f5c98001380 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721783871.851379    2129 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4080 Laptop GPU, Compute Capability 8.9


  2/473 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - class_loss: 12.3370 - regress_loss: 34.2503 - total_loss: 40.4188  

I0000 00:00:1721783879.759170    2129 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


473/473 ━━━━━━━━━━━━━━━━━━━━ 58s 86ms/step - class_loss: 0.5082 - regress_loss: 3.2608 - total_loss: 3.5150 - val_class_loss: 0.0427 - val_regress_loss: 0.9440 - val_total_loss: 0.9654
Epoch 2/20
473/473 ━━━━━━━━━━━━━━━━━━━━ 38s 66ms/step - class_loss: 0.0826 - regress_loss: 0.9181 - total_loss: 0.9594 - val_class_loss: 0.0105 - val_regress_loss: 0.5590 - val_total_loss: 0.5642
Epoch 3/20
473/473 ━━━━━━━━━━━━━━━━━━━━ 39s 67ms/step - class_loss: 0.1541 - regress_loss: 1.4525 - total_loss: 1.5296 - val_class_loss: 0.1062 - val_regress_loss: 2.0242 - val_total_loss: 2.0773
Epoch 4/20
473/473 ━━━━━━━━━━━━━━━━━━━━ 40s 68ms/step - class_loss: 0.0854 - regress_loss: 1.0080 - total_loss: 1.0507 - val_class_loss: 0.0132 - val_regress_loss: 0.4181 - val_total_loss: 0.4247
Epoch 5/20
473/473 ━━━━━━━━━━━━━━━━━━━━ 42s 73ms/step - class_loss: 0.0544 - regress_loss: 0.5770 - total_loss: 0.6042 - val_class_loss: 0.0110 - val_regress_loss: 0.2654 - val_total_loss: 0.2709
Epoch 6/20
473/473 ━━━━━━━━━━━━

In [87]:
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_class_loss'], color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'], color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()

In [ ]:
## testing the model

In [88]:
test_data = test.as_numpy_iterator()

In [89]:
test_sample = test_data.next()

In [90]:
yhat = facetracker.predict(test_sample[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step


In [91]:
fig, ax = plt.subplots(ncols=4, figsize=(20, 20))
for idx in range(4):
    sample_image = test_sample[0][idx]
    sample_coords = yhat[1][idx]
    
    # Create a writable copy of the sample_image
    sample_image_writable = np.copy(sample_image)

    if yhat[0][idx] > 0.5:
        cv2.rectangle(sample_image_writable, 
                      tuple(np.multiply(sample_coords[:2], [120, 120]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [120, 120]).astype(int)),
                      (255, 0, 0), 2)
    ax[idx].imshow(sample_image_writable)
plt.show() 

In [ ]:
### Save the model

In [1]:
from tensorflow.keras.models import load_model

In [93]:
facetracker.save('facetracker.h5')

In [2]:
facetracker = load_model('facetracker.h5')

I0000 00:00:1721785048.391310    9725 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1721785048.405290    9725 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1721785048.405323    9725 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1721785048.408803    9725 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1721785048.408833    9725 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [3]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import time
import tensorflow as tf

In [4]:
cap = cv2.VideoCapture('/dev/video0')

if not cap.isOpened():
    print("Error: Could not open webcam")
    exit()
    
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 15)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))

time.sleep(2)
    
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1721785057.040704    9907 service.cc:146] XLA service 0x7f62f0003cc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721785057.040737    9907 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4080 Laptop GPU, Compute Capability 8.9


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


I0000 00:00:1721785058.492100    9907 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
QObject::moveToThread: Current thread (0x55d39a462e10) is not the object's thread (0x55d39971a180).
Cannot move to target thread (0x55d39a462e10)

QObject::moveToThread: Current thread (0x55d39a462e10) is not the object's thread (0x55d39971a180).
Cannot move to target thread (0x55d39a462e10)

QObject::moveToThread: Current thread (0x55d39a462e10) is not the object's thread (0x55d39971a180).
Cannot move to target thread (0x55d39a462e10)

QObject::moveToThread: Current thread (0x55d39a462e10) is not the object's thread (0x55d39971a180).
Cannot move to target thread (0x55d39a462e10)

QObject::moveToThread: Current thread (0x55d39a462e10) is not the object's thread (0x55d39971a180).
Cannot move to target thread (0x55d39a462e10)

QObject::moveToThread: Current thread (0x55d39a462e10) is not the object's thread (0x55d39971a180).
Cannot mo

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


QObject::moveToThread: Current thread (0x55d39a462e10) is not the object's thread (0x55d39971a180).
Cannot move to target thread (0x55d39a462e10)

QObject::moveToThread: Current thread (0x55d39a462e10) is not the object's thread (0x55d39971a180).
Cannot move to target thread (0x55d39a462e10)



...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━

KeyboardInterrupt: 

In [ ]:
import cv2

# Check the first 10 indexes to find the available cameras
for index in range(10):
    cap = cv2.VideoCapture(index)
    if cap.read()[0]:
        print(f"Camera index {index} is working")
        cap.release()
    else:
        print(f"Camera index {index} is not available")